# Example Report: Distribution of train stops in germany

The question that interests us is: 

This report is based on open data from two different datasources.

#### Datasource 1

#### Datasource 2
The second datasource is the DB Timetable API Version 1.0.x. The timetables API can be used to query information about the current (train) traffic situation in Germany and its causes. For the report API endpoints are called that return a Timetable object that contains all known changes for a station given by an *eva number* (train station identifier). 

For further information see the official website (https://developers.deutschebahn.com/db-api-marketplace/apis/product/timetables) There you can find also the OpenAPI-document of the DB Timetable API.

The Timetables APIs data is provided under a [Creative Commons Attribution 4.0 International (CC BY 4.0)0](https://creativecommons.org/licenses/by/4.0/) license.

## Install dependencies
Initially, install all required dependencies. The specific version of SQLAlchemy is needed because SQLAlchemy 2.0 does not work with pandas yet. nbformat allows the use of the "notebook" formatter for the plot, others can not be rendered to HTML.

In [24]:
%pip install pandas
%pip install plotly
%pip install SQLAlchemy
%pip install nbformat
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Load data
Create a pandas dataframe using the local sqlite file.

In [25]:
import pandas as pd

ds1_df = pd.read_sql_table('connection_time_graph', 'sqlite:///project/data/train_connection_analysis.sqlite')
ds2_df = pd.read_sql_table('timetable_for_stations', 'sqlite:///project/data/train_connection_analysis.sqlite')

## Towns/Trainstops in Germany that are included in this analysis?
To show which towns/trainstops are included in our analysis, we use plotly to draw a scatterplot of all train stops in the dataset, overlaying it on a map from OpenStreetMap.

// The train stops will be colored based on the `Betreiber_Name`, allowing us to see what area an operator services.

In [26]:
import plotly.io as pio
import plotly.express as px

# pio.renderers.default = "notebook"

"""
fig = px.scatter_mapbox(df, 
                        lat="Breite", 
                        lon="Laenge", 
                        hover_name="NAME", 
                        hover_data=["EVA_NR", "DS100", "Betreiber_Name"],
                        color="Betreiber_Name",
                        zoom=5, 
                        height=800,
                        width=1200)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()"""

'\nfig = px.scatter_mapbox(df, \n                        lat="Breite", \n                        lon="Laenge", \n                        hover_name="NAME", \n                        hover_data=["EVA_NR", "DS100", "Betreiber_Name"],\n                        color="Betreiber_Name",\n                        zoom=5, \n                        height=800,\n                        width=1200)\n\nfig.update_layout(mapbox_style="open-street-map")\nfig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})\nfig.show()'

### Connection Times for a specific connection
To answer the question about a specific connection in detail, we make details for a specific connection accessible. At first we show the information as it is in the dataset. Then we will aggregate the data by towns and calculate metrics to compare train and car connections by town.

Just select your \<Source\> and your \<Destination\>. It will show the duration in minutes for a possible connection and the transportType of the connection.

Note: There are different train connections with different connection times.

In [89]:
import ipywidgets as widgets
from ipywidgets import interact

sources = list(ds1_df["source"].unique())
destinations = list(ds1_df["destination"].unique())

@interact
def show_basic_connection_information(source=sources,
                                destination=destinations):
    connection = ds1_df[(ds1_df["source"] == source) & (ds1_df["destination"] == destination)]
    connection = connection[["source", "destination", "duration", "transportType"]]
    connection["duration"] = pd.to_datetime(connection.duration, unit='m').dt.strftime('%Hh %Mmin')
    print(connection)

interactive(children=(Dropdown(description='source', options=('Aachen', 'Augsburg', 'Berlin', 'Bielefeld', 'Bo…

### Aggregate connection times per connection into a single row and calculate mean/min train duration

To answer the question where are already good train connections, we need to compare the connection times by car with them by train. Therefore we pick the fastest train connection and the median and compare connection times to the ones by car. We store the information for a specific connection in a single row

In [57]:
# Select the train connections, group the trains connections by 'source' and 'destination' and calculate the median and minimum duration
train_df = ds1_df[ds1_df['transportType'] == 'train']
train_grouped = train_df.groupby(['source', 'destination'])['duration'].agg(['median', 'min']).reset_index()

# Filter the DataFrame for 'car' durations
car_df = ds1_df[ds1_df['transportType'] == 'car']
car_df = car_df[["source", "destination", "duration"]]
#print(car_df)

# Merge the train_df and car_df on 'source' and 'destination'
connection_times_df = pd.merge(train_grouped, car_df, on=['source', 'destination'], how='left')

# Rename the columns
connection_times_df.rename(columns={'median': 'median_train_duration', 'min': 'min_train_duration', 'duration': 'car_duration'}, inplace=True)

print(connection_times_df.head(2))

   source destination  median_train_duration  min_train_duration  car_duration
0  Aachen    Augsburg                  314.0                 302           318
1  Aachen      Berlin                  356.0                 351           352


### Compare train connection times with car connection times
To answer the question where a train connection is already better than a car connection, calculate the difference of the connection times. 

* Positive values x mean that a train connection is faster by x minutes than the car connection between a source and a destination.
* Negative values x mean that a train connection is slower by x minutes than the car connection between a source and a destination.

In [96]:
connection_times_df["diff_car_median_train_duration"] = connection_times_df["car_duration"] - connection_times_df["median_train_duration"]
connection_times_df["diff_car_min_train_duration"] = connection_times_df["car_duration"] - connection_times_df["min_train_duration"]

print(connection_times_df.head(2))

   source destination  median_train_duration  min_train_duration  \
0  Aachen    Augsburg                  314.0                 302   
1  Aachen      Berlin                  356.0                 351   

   car_duration  diff_car_median_train_duration  diff_car_min_train_duration  
0           318                             4.0                           16  
1           352                            -4.0                            1  


### Show metrics for a specific connection

Now we can show the calculated metrics for a specific connection and show which transportation type is faster for a specific connection between two towns.

In [101]:
# remember, we calculated the sources and destinations list for the dropbox earlier

@interact
def show_metrics_for_a_connection(source=sources,
                                destination=destinations):
    connection = connection_times_df[(connection_times_df["source"] == source) & (connection_times_df["destination"] == destination)]
    print(connection)
    diff_car_min_train_duration = connection["diff_car_min_train_duration"].values[0]
    diff_car_median_train_duration = connection["diff_car_median_train_duration"].values[0]
    
    if diff_car_min_train_duration > 0:
        print(f"\nThe fastest train connection from {source} to {destination} is {diff_car_min_train_duration} minutes faster than the car connection.")
    else:
        print(f"\nThe fastest train connection from {source} to {destination} is {diff_car_min_train_duration} minutes slower than the car connection.")
    
    if diff_car_median_train_duration > 0:
        print(f"\nThe median train connection from {source} to {destination} is {diff_car_median_train_duration} minutes faster than the car connection.")
    else:
        print(f"\nThe median train connection from {source} to {destination} is {diff_car_median_train_duration} minutes slower than the car connection.")

interactive(children=(Dropdown(description='source', options=('Aachen', 'Augsburg', 'Berlin', 'Bielefeld', 'Bo…

## Ranking of towns with good train connections

To show which towns already have good train connections we now calculate for all outgoing connections from a town if the car or the train is faster to all destinations and count the results. We then create a ranking to highlight towns that are better accessible by car and towns that are better accessible by train.

In [132]:
min_train_faster = connection_times_df.groupby("source")["diff_car_min_train_duration"].apply(lambda diff_car_min_train_duration: (diff_car_min_train_duration > 0).sum()).reset_index(name="min_train_faster")
median_train_faster = connection_times_df.groupby("source")["diff_car_median_train_duration"].apply(lambda diff_car_min_train_duration: (diff_car_min_train_duration > 0).sum()).reset_index(name="median_train_faster")

town_ranking = pd.merge(min_train_faster, median_train_faster, on="source")

Towns sorted by the number of outgoing connections where the **fastest** train connection is faster than the car connection:

In [129]:
town_ranking.sort_values(by=["min_train_faster"], inplace=True, ascending=False)
print(town_ranking)

                    source  min_train_faster  median_train_faster
2                   Berlin                57                   26
44                Mannheim                47                   38
64               Stuttgart                43                   24
1                 Augsburg                42                   18
33               Karlsruhe                40                   11
..                     ...               ...                  ...
35                    Kiel                 1                    0
68  Villingen-Schwenningen                 0                    0
65                   Trier                 0                    0
19               Flensburg                 0                    0
37                Konstanz                 0                    0

[75 rows x 3 columns]


Towns sorted by the number of outgoing connections where the **median** train connection is faster than the car connection:

In [131]:
town_ranking.sort_values(by=["median_train_faster"], inplace=True, ascending=False)
print(town_ranking)

                    source  min_train_faster  median_train_faster
44                Mannheim                47                   38
2                   Berlin                57                   26
64               Stuttgart                43                   24
15                Duisburg                32                   21
30              Ingolstadt                35                   20
..                     ...               ...                  ...
65                   Trier                 0                    0
68  Villingen-Schwenningen                 0                    0
35                    Kiel                 1                    0
19               Flensburg                 0                    0
37                Konstanz                 0                    0

[75 rows x 3 columns]


## Delays in the operation of a specific train station
As for connections also the information of Datasource2 should be able to be filtered. Just select your \<Station\> and a \<Delay cause\> to make delay causes visible for a specific station

In [29]:
ZUGINFO_NRW = ". (Quelle: zuginfo.nrw)"

stations = list(ds2_df["train_station"].unique())
ds2_df["category"] = ds2_df["category"].replace(ZUGINFO_NRW, "", regex=True)
delay_causes = list(ds2_df["category"].unique())


@interact
def show_train_station_information(train_station=stations, delay_cause=delay_causes):
    station = ds2_df[(ds2_df["train_station"] == train_station) & (ds2_df["problems_found"] == True)]
    if(station.empty):
        print("No problems for station ", train_station, " found!")
    else:
        station = station[["train_station", "from_time", "to_time", "category", "priority"]]
        station = station[station["category"] == delay_cause]
        station["from_time"] = pd.to_datetime(station["from_time"], format='%y%m%d%H%M')
        station["to_time"] = pd.to_datetime(station["to_time"], format='%y%m%d%H%M')
        station["duration"] = station["to_time"] - station["from_time"]
        print(station)

interactive(children=(Dropdown(description='train_station', options=('Aachen Hbf', 'Augsburg Hbf', 'Berlin Hbf…

<Destination/>